<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao27bis_B2W_csv_sentiment_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Colab: Fine-Tuning do BERT (Português) para Análise de Sentimentos (B2W-Reviews)

Este notebook demonstra o processo de fine-tuning de um modelo BERT pré-treinado para a língua portuguesa (`neuralmind/bert-base-portuguese-cased`) para a tarefa de classificação de sentimentos.

Usaremos o dataset `b2w-reviews`, que contém reviews de produtos em português.

## Passo 1: Instalação e Configuração

Primeiro, instalamos as bibliotecas necessárias.

In [1]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00


In [2]:
import string
import pandas as pd
# Para pré-processamento
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
from sklearn.model_selection import train_test_split


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# Importações principais
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
import evaluate

In [4]:
!git clone https://github.com/americanas-tech/b2w-reviews01.git

Cloning into 'b2w-reviews01'...
remote: Enumerating objects: 10, done.
remote: Total 10 (delta 0), reused 0 (delta 0), pack-reused 10 (from 1)
Receiving objects: 100% (10/10), 19.82 MiB | 18.42 MiB/s, done.


In [5]:
# Célula 2: Carregar Dados da B2W-Reviews de uma URL

def carregar_dados_b2w():
    """Carrega o dataset B2W-Reviews a partir de uma URL e trata os labels."""

    # URL de um Gist com o dataset (pode levar um momento para carregar)
    url = "b2w-reviews01/B2W-Reviews01.csv"

    print(f"Carregando dados de: {url}")
    df = pd.read_csv(url, sep=',')

    # Remove colunas desnecessárias e linhas com dados faltantes
    df = df[['review_text', 'overall_rating']].dropna()

    # --- Mapeamento de Labels (Rating -> Sentimento) ---
    # 1. Remove reviews neutras (rating == 3)
    df_filtered = df.query("overall_rating != 3").copy()

    # 2. Mapeia 1-2 estrelas para 0 (Negativo) e 4-5 estrelas para 1 (Positivo)
    df_filtered['sentiment'] = df_filtered['overall_rating'].apply(lambda x: 1 if x > 3 else 0)

    # Seleciona apenas as colunas que vamos usar
    df_final = df_filtered[['review_text', 'sentiment']]

    return df_final

df = carregar_dados_b2w()

print("\nDados carregados e mapeados:")
print(f"Total de amostras: {len(df)}")
print("\nDistribuição de sentimentos:")
print(df['sentiment'].value_counts())

display(df.head())

Carregando dados de: b2w-reviews01/B2W-Reviews01.csv


/tmp/ipython-input-494068140.py:10: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, sep=',')



Dados carregados e mapeados:
Total de amostras: 113088

Distribuição de sentimentos:
sentiment
1    79316
0    33772
Name: count, dtype: int64


,review_text,sentiment
0,Estou contente com a compra entrega rápida o ú...,1
1,"Por apenas R$1994.20,eu consegui comprar esse ...",1
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1
4,"A entrega foi no prazo, as americanas estão de...",1


## Passo 2: Carregamento do Dataset (B2W-Reviews)

Carregamos o dataset `b2w-reviews`. Este dataset possui as colunas `review_text` (o texto) e `stars` (de 1 a 5).

In [6]:
# Célula 3: Função de Pré-processamento (Português)

# Obtém a lista de stopwords em PORTUGUÊS
stop_words = set(stopwords.words('portuguese'))

def preprocessar_texto(text):
    """
    Limpa o texto:
    1. Remove tags HTML (embora esta base não tenha muitas)
    2. Remove pontuação
    3. Converte para minúsculas
    4. Remove stopwords (em português)
    """
    if not isinstance(text, str):
        return ""

    # 1. Remove HTML
    text = re.sub(r'<[^>]+>', ' ', text)

    # 2. Remove pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))

    # 3. Converte para minúsculas
    text = text.lower()

    # 4. Remove stopwords
    palavras = [palavra for palavra in text.split() if palavra not in stop_words]

    return ' '.join(palavras)

# Aplica o pré-processamento
# (Pode levar alguns minutos)
print("Iniciando pré-processamento...")
df['clean_text'] = df['review_text'].apply(preprocessar_texto)
print("Pré-processamento concluído.")

display(df.head())

Iniciando pré-processamento...
Pré-processamento concluído.


,review_text,sentiment,clean_text
0,Estou contente com a compra entrega rápida o ú...,1,contente compra entrega rápida único problema ...
1,"Por apenas R$1994.20,eu consegui comprar esse ...",1,apenas r199420eu consegui comprar lindo copo a...
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1,supera agilidade praticidade outras panelas el...
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1,filho amou parece verdade tantos detalhes têm
4,"A entrega foi no prazo, as americanas estão de...",1,entrega prazo americanas parabéns smart tv boa...


In [7]:
# Célula 4: Divisão de Treino e Teste


# Como este dataset não tem um split padrão, vamos criar um
# Usaremos 75% para treino e 25% para teste

X = df['clean_text']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

print(f"Tamanho do treino: {len(X_train)} amostras")
print(f"Tamanho do teste: {len(X_test)} amostras")

Tamanho do treino: 84816 amostras
Tamanho do teste: 28272 amostras


In [8]:
from datasets import Dataset

# Convert pandas DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(pd.DataFrame({'text': X_train.tolist(), 'label': y_train.tolist()}))
test_dataset = Dataset.from_pandas(pd.DataFrame({'text': X_test.tolist(), 'label': y_test.tolist()}))

print("Hugging Face Datasets criados:")
print(train_dataset)
print(test_dataset)

Hugging Face Datasets criados:
Dataset({
    features: ['text', 'label'],
    num_rows: 84816
})
Dataset({
    features: ['text', 'label'],
    num_rows: 28272
})


## Passo 5: Carregamento do Tokenizador e Modelo (Português)

Agora, usamos o `neuralmind/bert-base-portuguese-cased` como nosso modelo base.

* **`cased`**: Significa que o modelo diferencia maiúsculas de minúsculas.
* **`id2label`**: Mapeia os IDs das labels (0 e 1) para nomes legíveis ("Negativo", "Positivo"). Isso melhora a saída do `pipeline`.

In [9]:
# Checkpoint do modelo em português
model_checkpoint = "neuralmind/bert-base-portuguese-cased"

# Carrega o tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)



# Carrega o modelo de classificação
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# --- INÍCIO DA MODIFICAÇÃO (CONGELAMENTO) ---

# 1. Congela todos os parâmetros do corpo do BERT
# (O nome 'bert' é específico para modelos BERT.
# Para RoBERTa seria 'roberta', para DistilBERT seria 'distilbert')
for param in model.bert.parameters():
    param.requires_grad = True

# 2. (Opcional, mas boa prática)
# Garantir que os parâmetros da cabeça de classificação ESTÃO treináveis.
# Eles já vêm com requires_grad=True por padrão, pois são novos.
for param in model.classifier.parameters():
    param.requires_grad = True

# --- FIM DA MODIFICAÇÃO ---

## Passo 6: Pré-processamento (Tokenização)

Esta função aplica o tokenizador aos nossos textos.

In [11]:
def tokenize_function(examples):
    # 'truncation=True' corta textos maiores que o limite máximo do modelo (512 para o BERT)
    # Explicitly add padding here when batched=True to ensure consistent shapes for the datasets library
    return tokenizer(examples["text"], truncation=True, max_length=512)

# Filter out examples with None values in 'text' before mapping
train_dataset_filtered = train_dataset.filter(lambda example: example["text"] is not None)
test_dataset_filtered = test_dataset.filter(lambda example: example["text"] is not None)

# Apply tokenization to the filtered datasets
tokenized_train_dataset = train_dataset_filtered.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset_filtered.map(tokenize_function, batched=True)


# The DataCollator will handle padding dynamically (more efficient)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Filter:   0%|          | 0/84816 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Filter:   0%|          | 0/28272 [00:00<?, ? examples/s]

Map:   0%|          | 0/84816 [00:00<?, ? examples/s]

Map:   0%|          | 0/28272 [00:00<?, ? examples/s]

## Passo 7: Definição da Métrica de Avaliação

Usaremos a Acurácia como métrica principal.

In [12]:
# Carrega a métrica de acurácia
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

## Passo 8: Configuração do Treinamento (Fine-Tuning)

Configuramos os `TrainingArguments` e o `Trainer`.

In [13]:
# Define os argumentos do treinamento
training_args = TrainingArguments(
    output_dir="./bert-b2w-finetuned",          # Onde salvar o modelo
    learning_rate=2e-5,                        # Taxa de aprendizado
    per_device_train_batch_size=8,             # Tamanho do batch de treino reduzido
    per_device_eval_batch_size=8,              # Tamanho do batch de avaliação reduzido
    num_train_epochs=3,                        # Número de épocas
    weight_decay=0.01,
    eval_strategy="epoch",               # Avaliar a cada época
    save_strategy="epoch",                     # Salvar a cada época
    load_best_model_at_end=True,               # Carregar o melhor modelo no final
    push_to_hub=False,
    # Adicionar gradient_accumulation_steps se reduzir o batch size não for suficiente
    gradient_accumulation_steps=4, # Example: Accumulate gradients over 2 steps
)

# Cria o objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

## Passo 9: Iniciar o Treinamento

**Certifique-se de estar em um ambiente com GPU!** (Vá em *Ambiente de execução > Alterar tipo de ambiente de execução > Acelerador de hardware > T4 GPU*).

In [14]:
# Inicia o fine-tuning!
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jsansao to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.141100,0.139183,0.949066
2,0.095400,0.140525,0.956246
3,0.070700,0.167261,0.956883


TrainOutput(global_step=7953, training_loss=0.11037732237886966, metrics={'train_runtime': 2858.2262, 'train_samples_per_second': 89.023, 'train_steps_per_second': 2.782, 'total_flos': 6438150746918880.0, 'train_loss': 0.11037732237886966, 'epoch': 3.0})

## Passo 10: Avaliação Final

Vejamos a acurácia final do nosso melhor modelo no conjunto de teste.

In [15]:
# Avalia o melhor modelo no dataset de teste
eval_results = trainer.evaluate()

print("\nResultados da Avaliação Final:")
print(eval_results)


Resultados da Avaliação Final:
{'eval_loss': 0.13918325304985046, 'eval_accuracy': 0.9490662139219015, 'eval_runtime': 89.2372, 'eval_samples_per_second': 316.819, 'eval_steps_per_second': 39.602, 'epoch': 3.0}


## Passo 11: Teste com Novas Frases (Inferência)

Vamos usar a ferramenta `pipeline` para testar nosso modelo recém-treinado com frases em português.

In [16]:
from transformers import pipeline

# Detecta se há GPU disponível
device = 0 if torch.cuda.is_available() else -1

# Carrega o pipeline de análise de sentimentos com nosso melhor modelo salvo
best_model_path = trainer.state.best_model_checkpoint
print(f"\nCarregando o melhor modelo salvo de: {best_model_path}")

sentiment_pipeline_pt = pipeline(
    "sentiment-analysis",
    model=best_model_path,
    tokenizer=tokenizer,
    device=device  # -1 para CPU, 0 para GPU
)

print("\n--- Teste de Inferência em Português ---")

review_pos = "Este produto é maravilhoso! Superou minhas expectativas, a entrega foi rápida e o material é de ótima qualidade."
review_neg = "Péssimo"
review_neu = "É um celular ok, funciona, mas a bateria não dura muito. Esperava mais pelo preço."

print(f"\nFrase: {review_pos}")
print(f"Resultado: {sentiment_pipeline_pt(review_pos)}")

print(f"\nFrase: {review_neg}")
print(f"Resultado: {sentiment_pipeline_pt(review_neg)}")

print(f"\nFrase: {review_neu}")
print(f"Resultado: {sentiment_pipeline_pt(review_neu)}")

Device set to use cuda:0



Carregando o melhor modelo salvo de: ./bert-b2w-finetuned/checkpoint-2651

--- Teste de Inferência em Português ---

Frase: Este produto é maravilhoso! Superou minhas expectativas, a entrega foi rápida e o material é de ótima qualidade.
Resultado: [{'label': 'LABEL_1', 'score': 0.9978140592575073}]

Frase: Péssimo
Resultado: [{'label': 'LABEL_0', 'score': 0.9741508960723877}]

Frase: É um celular ok, funciona, mas a bateria não dura muito. Esperava mais pelo preço.
Resultado: [{'label': 'LABEL_0', 'score': 0.8960380554199219}]
